<a href="https://colab.research.google.com/github/minerado/ml/blob/main/gpt-implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>